## Getting Embeddings and Testing Responses

In [8]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from yandex_chain import YandexEmbeddings
from langchain.vectorstores import FAISS
from yandex_chain import YandexLLM
from langchain.chains import RetrievalQA

##### Loading text files and splitting them into chunks

In [2]:
loader1 = TextLoader("data/ai_parced.txt", encoding="utf-8")
loader2 = TextLoader("data/ai_product_parced.txt", encoding="utf-8")

docs = loader1.load() + loader2.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

In [4]:
api_key = "***"
folder_id = "***"

In [ ]:
embeddings = YandexEmbeddings(api_key=api_key, folder_id=folder_id)

##### Creating a vector store

In [5]:
# vectorstore = FAISS.from_documents(split_docs, embeddings)
#
# vectorstore.save_local('data/ai_programs')

##### Loading the vector store from disk

In [7]:
vectorstore = FAISS.load_local(folder_path='data/ai_programs', embeddings=embeddings, allow_dangerous_deserialization=True)

In [14]:
instruction_text = 'Ты интеллектуальный помощник, который помогает абитуриентам выбрать одну из двух программ магистратуры: Искусственный Интеллект (AI) или Управление ИИ Продуктами (AI Product Management). Получай информацию о программах из файлов. Отвечай на вопросы пользователя по поводу выбора образовательных программ. Игнорируй вопросы не связанные с обучением в магистратуре.'

In [15]:
llm = YandexLLM(api_key=api_key, folder_id=folder_id, instruction_text=instruction_text)

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    return_source_documents=True
)

##### Testing the model with a query

In [17]:
query = "Я хочу заниматься современными языковыми моделями. Какую программу мне посоветуешь?"
result = qa_chain.invoke({"query": query})

In [18]:
print("Ответ:", result["result"])

Ответ: Программа «Искусственный Интеллект (AI)» включает курсы по глубокому обучению и обработке естественного языка, что может быть полезно для изучения современных языковых моделей.

Источники:
- data/ai_product_parced.txt
- data/ai_parced.txt
- data/ai_parced.txt
